## Get experimental information from HTE PPT and Excel files

        Ronghui Zhou, rzhou11@its.jnj.com
        July 16, 2020

In [1]:
# import libraries
import os
import re
import glob
import numpy as np
import pandas as pd
from pptx import Presentation

## PART 1. Get experimental information from HTE PPT files

In [2]:
# define a funciton to extract all text from pptx files
def get_text_from_ppt(ppt):
    """
    Get text from HTE pptx files
    
    INPUT: pptx filepath, or filename if in the same directory
    
    OUTPUT: a list of text string
    
    """
    
    # define a list to hold all text from the pptx
    exp_text = []    
    
    # initiate the Presentaiton function
    prs = Presentation(ppt)

    # loop through all slides and look for texts, append to exp_text list
    for slide in prs.slides:        
        slide_text =[]        
        for shape in slide.shapes:
            if hasattr(shape, 'text'):
                slide_text.append(shape.text)
        exp_text.append(slide_text)
    
    return exp_text


###########################################################################################################################
# define a funciton to extract 'HTE Chemist', 'Collaborator', 'Project', 'Date', 'Yield' and 'Comments' from text_from_ppt
def get_exp_info_from_text(text_from_ppt):
    """
    Get 'HTE Chemist', 'Collaborator', 'Project', 'Date', 'Yield' and 'Comment' information from text_from_ppt 
        (a list of strings from the ppt file)
    
    INPUT: text_from_ppt (a list of strings from the ppt file), 
            two pages: page 1, background, issue and previous work; page 2, results
    
    OUTPUT: 'HTE Chemist', 'Collaborator', 'Project', 'Date', 'Yield' and 'Comment'
    
    """
    # get chemist, collaborator, project and date
    HTE_chemist = [text for text in text_from_ppt[0] if 'HTE Chemist:' in text][0].split(':')[1].replace('\xa0','').strip()
    collaborator = [text for text in text_from_ppt[0] if 'Collaborator:' in text][0].split(':')[1].replace('\xa0','').strip()
    project = [text for text in text_from_ppt[0] if 'Project:' in text][0].split(': ')[1].replace('\xa0','').strip()
    exp_date = [text for text in text_from_ppt[0] if 'Date:' in text][0].split(':')[1].replace('\xa0','').strip()
    
    # slice the strings containing 'yield'                
    yield_text = [text for text in text_from_ppt[1] if 'yield' in text.lower()]
    
    try:
        # find the yield value which ends with '%'
        yield_val = re.findall(r'\d+%', ' '.join(yield_text))[0]
    except:
        # assign yield = 0 if not available
        yield_val = 0
    
    # extract comments
    comment = [text for slide in text_from_ppt for text in slide if 'comment' in text.lower()]

    try:
        comment = comment[0].replace('Comments:',' ').replace('\n',' ').rstrip().lstrip()
    except:
        comment = 'NA' # if there is no comment in the ppt, assign it as 'NA'
    
    return HTE_chemist, collaborator, project, exp_date, yield_val, comment


###########################################################################################################################
# define a function to print experimental information
def print_exp_info(ppt, to_print=False):
    """
    print experimental information
    INPUT: 
        ppt file path, or filename if in the same directory
        to_print: whether to print or not
    
    """
    # get all text fromt the ppt file
    text_from_ppt = get_text_from_ppt(ppt)
    
    # get selected key information
    HTE_chemist, collaborator, project, exp_date, yield_val, comment = get_exp_info_from_text(text_from_ppt)

    # if to_print is True, print full text and selected key information from the extracted information
    if to_print:
        print(f'\033[1mAll text extracted from "{ppt}":\033[0m \n{text_from_ppt}.')
        print(f'\n\033[1mSelected key information: \033[0m\nFilename:\t{ppt}')
        print(f'HTE Chemist: \t{HTE_chemist} \nCollaborator: \t{collaborator}\
              \nProject: \t{project} \nDate: \t\t{exp_date} \nYield: \t\t{yield_val}\
              \n\033[1mComments:\033[0m \t{comment}')    

### Example

In [3]:
# define ppt filename and print experimental information
ppt = r'\\rndusshnas02\ds_disc_chem\HTE\rzhou11\get_exp_from_ppt_xlsx\2019_11_05 - jcompto4_2 - DGAT2 - Tudge Photoredox Minisci .pptx'    
print_exp_info(ppt, to_print=True)

All text extracted from "\\rndusshnas02\ds_disc_chem\HTE\rzhou11\get_exp_from_ppt_xlsx\2019_11_05 - jcompto4_2 - DGAT2 - Tudge Photoredox Minisci .pptx": 
[['Reaction test based on DiRocco, D. A.; Dykstra, K.; Krska, S.; Vachal, P.; Conway, D. V.; Tudge, M., Angew. Chem. Int. Ed. 2014, 53 (19), 4802-4806.\n\n Using excess radical precursor gave a complex mixture of mono-di, and trimethylated products.  \n\nRegioselectivity is poor', 'HTE Collaboration', 'Reaction Scheme', 'Collaborator:', 'Date: 11/4/2019', 'Plate design planning (mark with an “x”)', '', '', '', '', 'Previous work/ Issues', '', 'Pre-dosed end-user kit\nCustom plate design (fill table below)\nCollaborator recommendations (fill table below)', '24\n48\n96', 'x', 'Comments:\nA 24 well plate was run using blue LED Lumidox irradiation. The reaction was run for 12 hours.  ', 'HTE Chemist: Chuck Hendrick, Jordie Compton', 'Project: DGAT2', 'x', 'Number of wells (mark with an “x”)'], ['HTE Work and Results', 'Reaction Scheme', 

### Build a dataframe for all ppt files

In [4]:
# get the list of pptx files in the specified directory
def get_ppt_lst(ppt_path):
    ppt_lst = glob.glob(ppt_path)
#     print(f'\033[1mPPT files:\033[0m \n\t{ppt_lst}. ')
    return ppt_lst

###########################################################################################################################
def get_ELN_df(ppt_lst):
    data = []
    for i, ppt in enumerate(ppt_lst):

        # get all text fromt the ppt file
        text_from_ppt = get_text_from_ppt(ppt)

        # get selected key information
        HTE_chemist, collaborator, project, exp_date, yield_val, comment = get_exp_info_from_text(text_from_ppt)      
        
        ppt_file = ppt.rsplit('\\',1)[1]
        
        data.append((ppt_file, HTE_chemist, collaborator, project, exp_date, yield_val, comment))

    # define column names and build the dataframe
    column_names = ['PPT', 'HTE_Chemist', 'Collaborator', 'Project', 'Date', 'Yield', 'Comment']
    df = pd.DataFrame(data, columns = column_names)
    
    return df

###########################################################################################################################
def ppt_main():    
    ppt_dir = r'\\rndusshnas02\ds_disc_chem\HTE\rzhou11\get_exp_from_ppt_xlsx'
    ppt_path = os.path.join(ppt_dir, '*.pptx')

    ppt_lst = get_ppt_lst(ppt_path)

    df = get_ELN_df(ppt_lst)
    df.replace('', 'NA', inplace=True)
    
    return df

###########################################################################################################################
def save_df(df, result_csv):
    
    result_dir = r'\\rndusshnas02\ds_disc_chem\HTE\rzhou11\get_exp_from_ppt_xlsx\results'

    result_path = os.path.join(result_dir, result_csv)

    df.to_csv(result_path, index=False)

In [5]:
# main function for PPT files
ppt_main()

,PPT,HTE_Chemist,Collaborator,Project,Date,Yield,Comment
0,2019_11_05 - jcompto4_2 - DGAT2 - Tudge Photor...,"Chuck Hendrick, Jordie Compton",NA,DGAT2,11/4/2019,35%,A 24 well plate was run using blue LED Lumidox...
1,2020_07_01_emercad2_692_SIK_Pd CN.pptx,Eduardo Mercado,Kohki Nakafuku,SIK,2020-07-01,0,NA
2,hbian_965-BFL-1 Screen Tracking.pptx,"Compton, Jordan",Haiyan Bian (hbian_965),BFL-1,02/06/2020,30%,NA


## PART 2. Get experimental information from HTE Excel files

In [6]:
# get the list of xlsx files in the specified directory
def get_xlsx_lst(xlsx_path):
    xlsx_lst = glob.glob(xlsx_path)
#     print(f'\033[1mPPT files:\033[0m \n\t{ppt_lst}. ')
    return xlsx_lst

###########################################################################################################################
# read in a specific sheet from Excel and return a dataframe
def get_rxn_df(xlsx, sheet_name, skiprows=None, usecols=None):
    """
    Get xlsx dataframe from HTE xlsx files
    
    INPUT: xlsx filepath, or filename if in the same directory
    
    OUTPUT: xlsx dataframe
    
    """    
    rxn_df = pd.read_excel(xlsx, sheet_name = sheet_name, skiprows=skiprows, usecols=usecols)
    
    return rxn_df

###########################################################################################################################
# get the top reaction conditions based on certain criteria
def get_best_condition(rxn_des_df, top_n = 1):
    
    sort_by = 'Product Area / IS Area'
    
    rxn_des_df.sort_values(by = sort_by, ascending = False, inplace=True)
    
    best_condition = rxn_des_df.head(top_n).to_dict(orient = 'records')[0]
    
    return best_condition

###########################################################################################################################
# get key reaction information and return a pandas dataframe
def get_key_rxn_info(xlsx_lst): 
    
    key_rxn_info_lst = []
    
    for xlsx in xlsx_lst:
        
        start_df = get_rxn_df(xlsx, sheet_name = 'Start', usecols=[0,3,4,10,11]) 

        start_cols = ['Coupling Type', 'HTE Kit Number', 'Experiment Notebook Number', 'SM1 Name',\
                      'SMILES SM1', 'SM2 Name', 'SMILES SM2', 'SM2 Equivalents',\
                        'Base Equiv', 'Catalyst Name', 'Catalyst Loading (mol%)', 'SMILES PDT']

        rxn_dict = dict(zip(start_df.loc[:, 'Unnamed: 3'], start_df.loc[:, 'Unnamed: 4']))    
        sml_dict = dict(zip(start_df.loc[:, 'Unnamed: 10'], start_df.loc[:, 'Unnamed: 11']))

        rxn_type = start_df.columns[0]

        rxn_eln = rxn_dict['Experiment Notebook Number']

        rxn_kit_num = rxn_dict['HTE Kit Number']

        sm1 = rxn_dict['SM1 Name']
        sm1_sml = sml_dict['SMILES SM1']

        sm2 = rxn_dict['SM2 Name']
        sm2_sml = sml_dict['SMILES SM2']

        sm2_eq = rxn_dict['SM2 Equivalents']

        base_eq = rxn_dict['Base Equiv']

        cat = rxn_dict['Catalyst Name']

        cat_eq = "{:.0%}".format(rxn_dict['Catalyst Loading (mol%)'])

        prd = sml_dict['SMILES PDT'] 
        key_rxn_info = [rxn_type, rxn_kit_num, rxn_eln, sm1, sm1_sml, sm2, sm2_sml, sm2_eq, base_eq, cat, cat_eq, prd]


        rxn_des_cols = ['Vial', 'Catalyst.1', 'Base.1', 'Solvent', 'SM1 Area / IS Area',\
                        'Product Area / IS Area', 'approx. conversion from SM', 'approx. % yield product']
        rxn_des_df = get_rxn_df(xlsx, sheet_name = 'Reaction Design', skiprows = 1, usecols = rxn_des_cols).dropna()


        best_condition = get_best_condition(rxn_des_df, top_n = 1)      


        key_rxn_info = key_rxn_info + list(best_condition.values())

        key_rxn_info = tuple(key_rxn_info)

        
        key_rxn_info_lst.append(key_rxn_info)

    
    key_rxn_cols = start_cols + rxn_des_cols

    key_rxn_info_df = pd.DataFrame(key_rxn_info_lst, columns = key_rxn_cols)


    new_cols = ['Vial', 'Coupling Type', 'HTE Kit Number', 'Experiment Notebook Number', 'SM1 Name',\
                  'SMILES SM1', 'SM2 Name', 'SMILES SM2', 'SM2 Equivalents', 'Catalyst Name',\
                'Catalyst.1', 'Catalyst Loading (mol%)', 'Base.1', 'Base Equiv', 'Solvent',\
                'SMILES PDT', 'SM1 Area / IS Area', 'Product Area / IS Area',\
                'approx. conversion from SM', 'approx. % yield product']
    key_rxn_info_df_new = key_rxn_info_df[new_cols]
    
    return key_rxn_info_df_new

###########################################################################################################################
# the main function
def xlsx_main():
    xlsx_dir = r'\\rndusshnas02\ds_disc_chem\HTE\rzhou11\get_exp_from_ppt_xlsx'
    xlsx_path = os.path.join(xlsx_dir, '*.xlsx')

    xlsx_lst = get_xlsx_lst(xlsx_path)

    key_rxn_info_df = get_key_rxn_info(xlsx_lst)
    
    return key_rxn_info_df

In [7]:
# execute the main function
xlsx_main()

,Vial,Coupling Type,HTE Kit Number,Experiment Notebook Number,SM1 Name,SMILES SM1,SM2 Name,SMILES SM2,SM2 Equivalents,Catalyst Name,Catalyst.1,Catalyst Loading (mol%),Base.1,Base Equiv,Solvent,SMILES PDT,SM1 Area / IS Area,Product Area / IS Area,approx. conversion from SM,approx. % yield product
0,D3,Pd C-N Coupling,NaN,emercad2_692,emercad2_686_1,NC1=NN2C=CC(C3=CC(OC(F)F)=NC=C3OCC(C)(C)C#N)=C...,4-CHLORO-6-METHYLPYRIMIDINE,CC1=CC(Cl)=NC=N1,1.1,Pd Precatalysts,MorDalPhos Pd G3,9%,K3PO4,3,Dioxane,FC(F)OC(C=C1C2=CC3=CC(NC4=NC=NC(C)=C4)=NN3C=C2...,0.363775,1.344711,0.727668,1.006689
1,D5,Pd C-N Coupling,NaN,knakafuk_321,knakafuk_315_1,ClC1=CC([C@H](C2CC2)NC(CCC(F)(F)F)=O)=C(N=N1)Cl,amino-isoxazole,NC1=CON=C1,1.2,Pd Precatalysts,XantPhos Pd G4,9%,Cs2CO3,3,t-Amyl alcohol,ClC(N=N1)=C([C@H](C2CC2)NC(CCC(F)(F)F)=O)C=C1N...,0.000000,0.378434,1.000000,0.641408


### Example

In [8]:
xlsx = r'\\rndusshnas02\ds_disc_chem\HTE\rzhou11\get_exp_from_ppt_xlsx\2020_07_01_emercad2_692_SIK_Pd CN.xlsx'

start_df = get_rxn_df(xlsx, sheet_name = 'Start', usecols=[0,3,4,10,11])
start_df.head()

,Pd C-N Coupling,Unnamed: 3,Unnamed: 4,Unnamed: 10,Unnamed: 11
0,NaN,NaN,NaN,NaN,NaN
1,Please fill in green highlighted cells with yo...,Experiment Notebook Number,emercad2_692,Reaction Scheme:,NaN
2,NaN,HTE Kit Number,NaN,NaN,NaN
3,NaN,NaN,NaN,paste here,NaN
4,NaN,Reaction Information,NaN,NaN,NaN


In [9]:
rxn_des_cols = ['Vial', 'Catalyst.1', 'Base.1', 'Solvent', 'SM1 Area / IS Area', 'Product Area / IS Area', 'approx. conversion from SM', 'approx. % yield product']

rxn_des_df = get_rxn_df(xlsx, sheet_name = 'Reaction Design', skiprows = 1, usecols = rxn_des_cols).dropna()

rxn_des_df.head()

,Vial,Catalyst.1,Base.1,Solvent,SM1 Area / IS Area,Product Area / IS Area,approx. conversion from SM,approx. % yield product
0,A1,P(tBu)3 Pd G4,Cs2CO3,Dioxane,1.257551,0.088334,0.058562,0.066129
1,A2,SPhos Pd G4,Cs2CO3,Dioxane,0.602642,1.120986,0.548845,0.839202
2,A3,tBuXPhos Pd G3,Cs2CO3,Dioxane,1.109260,0.119171,0.169576,0.089215
3,A4,RuPhos Pd G4,Cs2CO3,Dioxane,0.589434,1.196884,0.558733,0.896021
4,A5,BrettPhos Pd G4,Cs2CO3,Dioxane,1.266953,0.289507,0.051523,0.216733
